In [1]:
def load_namespace():
    import sys
    sys.path.insert(1,f'/wsu/home/gy/gy40/gy4065/hm_jetscapeml_source')#WSU Grid
    sys.path.insert(1,'/content/drive/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')#Colab GDrive v1
    sys.path.insert(1,'/content/drive/MyDrive/Projects/110_JetscapeMl/hm_jetscapeml_source')#Colab GDrive v2
    sys.path.insert(1,f'/mnt/g/My Drive/Projects/110_JetscapeMl/hm_jetscapeml_source')#wsl gdrive
    sys.path.insert(1,'G:\\My Drive\\Projects\\110_JetscapeMl\\hm_jetscapeml_source') #Windows GDrive
    sys.path.insert(1,'/home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/') #office tower
    
load_namespace()

In [2]:
# Loading/Preparing Environment for simulation
from jet_ml.config import Config
folds=5
epochs=2
dataset_size=1000 #10800000 #1000000
model_name="res_net"
simulation_name=f"alpha_s_{model_name}_{folds}_fold_{epochs}_epoch_{int(dataset_size/1000)}k_dataset_size"

config=Config(simulation_name=simulation_name)
config.__str__()

2024-08-28 10:26:31.534539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-28 10:26:31.541861: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-28 10:26:31.550738: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-28 10:26:31.553368: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-28 10:26:31.559877: I tensorflow/core/platform/cpu_feature_guar

Directory /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/models/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.
Directory /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.
Directory /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports/figures/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size already exists.


I0000 00:00:1724855192.632684  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


'Project Root: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source\nData Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/data\nModels Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/models\nReports Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports\nFigures Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports/figures\nSimulation Models Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/models/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nSimulation Reports Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nSimulation Figures Directory: /home/arsalan/Projects/110_JetscapeML/hm_jetscapeml_source/reports/figures/alpha_s_res_net_5_fold_2_epoch_1k_dataset_size\nEnvironment Details:\n  TensorFlow Version: 2.17.0\n  Keras Version: 3.5.0\n  Python Version: 3.12.5 | packaged by conda-forge | (main, 

I0000 00:00:1724855192.654803  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724855192.654933  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355


In [3]:
from IPython.display import display
# with tf.device("CPU"):
import jet_ml.classifiers.alpha_s.preprocess_dataset as pred
(x,y_raw,y_df)=pred.preprocess_dataset_for_alpha_s(dataset_size)
y_classes=y_df.columns
y=y_df.values
display("y_classes: ",y_classes)
display("y: ",y[:5])
display("y_raw: ",y_raw[:5])

Dataset Preprocessor
Loading the whole dataset
Extract the working column#1 for classification
dataset.x: <class 'numpy.ndarray'> 1024000 (1000, 32, 32)
dataset.y: <class 'numpy.ndarray'> 1000 (1000,)


'y_classes: '

Index(['0.2', '0.3', '0.4'], dtype='object')

'y: '

array([[0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1]])

'y_raw: '

array(['0.4', '0.4', '0.2', '0.4', '0.4'], dtype='<U32')

In [4]:
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(x) * TRAIN_PCT)

df_train_cut = x[0:TRAIN_CUT]
df_validate_cut = x[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")

Training size: 900
Validate size: 100


In [ ]:
#checking data is normalize
import numpy as np
x_max=np.max(x)
display(x_max)

In [12]:
display(x.shape)
display(y.shape)

(1000, 32, 32, 1)

(1000, 3)

In [24]:
# Transpose and then rearrange the dimensions
y_transposed = np.transpose(y)  # Shape will be (3, 1000)

# y_transposed is now (3, 1000)
print("Transposed shape:", y_transposed.shape)

# Optionally, you might want to add an extra dimension if needed
y_final = np.expand_dims(y, axis=2)  # Shape will be (1, 3, 1000)

# Check the final shape
print("Final shape:", y_final.shape)  # Should be (1, 3, 1000)

Transposed shape: (3, 1000)
Final shape: (1000, 3, 1)


In [5]:
WIDTH = 256
HEIGHT = 256

In [ ]:
from jet_ml.classifiers.alpha_s.preprocess_dataset import get_preprocess_dataset_info
get_preprocess_dataset_info(x,y)

In [6]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Convert grayscale to RGB (if needed)
x_rgb = np.concatenate([x] * 3, axis=-1)  # Convert to RGB

# Resize images to the target size
TARGET_WIDTH = 256
TARGET_HEIGHT = 256
with tf.device("CPU"):
    x_resized = np.array([tf.image.resize(image, [HEIGHT, WIDTH]) for image in x_rgb])


I0000 00:00:1724855216.530397  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724855216.530555  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724855216.530606  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1724855216.575222  430373 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [25]:
# Define ImageDataGenerator
datagen = ImageDataGenerator(
    # rescale=1./255,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create the data generator
batch_size = 32

train_generator = datagen.flow(
    x_resized,
    y_final,
    batch_size=batch_size,
    shuffle=True,
    seed=42  # For reproducibility
)

In [ ]:
datagen.flow_from_dataframe()

In [26]:

# Define a simple model with the updated input shape
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

with tf.device("CPU"):
    # Train the model using the data generator
    model.fit(train_generator, epochs=10, steps_per_epoch=len(x_resized) // batch_size)

Epoch 1/10


/home/arsalan/miniconda3/envs/tensorflow/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


ValueError: Argument `output` must have rank (ndim) `target.ndim - 1`. Received: target.shape=(None, 3), output.shape=(None, 3)

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator


training_datagen = ImageDataGenerator(
  rescale = 1./1,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

In [ ]:
type(x)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the custom data generator class
class NumpyArrayDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, x_data, y_data, batch_size, datagen, target_size, shuffle=True):
        self.x_data = x_data
        self.y_data = y_data
        self.batch_size = batch_size
        self.datagen = datagen
        self.target_size = target_size

        self.shuffle = shuffle
        self.indexes = np.arange(len(self.x_data))
        self.on_epoch_end()
        
    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.x_data) / self.batch_size))
    
    def __getitem__(self, index):
        # Generate one batch of data
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x = self.x_data[indexes]
        batch_y = self.y_data[indexes]
        
        # Resize images to target size
        resized_images = [tf.image.resize(img, [self.target_size[0], self.target_size[1]]) for img in batch_x]
        resized_images = np.array(resized_images)
        
        # Apply data augmentation
        augmented_images = []
        for img in batch_x:
            img = np.expand_dims(img, axis=0)
            augmented_img = self.datagen.flow(img, batch_size=1).next()[0]
            augmented_images.append(augmented_img)
        
        return np.array(augmented_images), np.array
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)

In [ ]:
# Instantiate the data generator
batch_size = 32
train_generator = NumpyArrayDataGenerator(x, y, batch_size, training_datagen,target_size=(HEIGHT, WIDTH),shuffle=False)

In [ ]:
type(train_generator)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# Define a simple model with the updated input shape
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(HEIGHT, WIDTH,1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model using the data generator
model.fit(train_generator, epochs=10)

In [ ]:
training_datagen.flow_from_dataframe(dataframe=df_train_cut,x_col='events',y_col=y,target_size=(HEIGHT,WIDTH),batch_size=32,class_mode='raw')

In [ ]:
train_generator = training_datagen.flow_from_dataframe(
        dataframe=df_train_cut,
        directory=x,
        x_col="filename",
        y_col="clip_count",
        target_size=(HEIGHT, WIDTH),
        # Keeping the training batch size small 
        # USUALLY increases performance
        batch_size=32, 
        class_mode='raw')

In [ ]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

WIDTH = 256
HEIGHT = 256

training_datagen = ImageDataGenerator(
  rescale = 1./255,
  horizontal_flip=True,
  #vertical_flip=True,
  fill_mode='nearest')

In [ ]:
# fold, shuffle, x, y_raw
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics
k_fold=StratifiedKFold(folds,shuffle=False)

out_of_sample_y=[]
out_of_sample_pred=[]
fold=0
folds_accuracy=[]
epochs_needed = []
times_taken=[]
#Must specify y StratifiedKFold for classification
for train,test in k_fold.split(x,y_raw):
    fold+=1
    print(f"Fold #{fold}")

    x_train=x[train]
    y_train=y[train]
    x_test=x[test]
    y_test=y[test]

    from jet_ml.models import vgg16_net
    input_shape=x[0].shape
    output_shape=y.shape[1]
    activation='softmax'
    # HIDE OUTPUT
    from tensorflow.keras.applications import MobileNet
    model = MobileNet(weights='imagenet',include_top=True)
    model.summary()
    model=vgg16_net.build_model(input_shape,output_shape,activation)
    model=vgg16_net.compile_model(model)


    batch_size=128
    # batch_size=256 #original patch size applies for eloss classification
    monitor='val_loss' #'val_accuracy' or 'val_loss'
    
    model, history,elapsed_time,stopped_epoch=vgg16_net.train_model(model,
                                        x_train,y_train, x_test,y_test, 
                                        epochs, batch_size, monitor,
                                        fold=fold)

    from jet_ml.evaluation import get_accuracy
    pred, score=get_accuracy(model,x_test=x_test,y_test=y_test)
    
    folds_accuracy.append(score)
    times_taken.append(elapsed_time)    
    epochs_needed.append(epochs)

    out_of_sample_y.append(y_test)
    out_of_sample_pred.append(pred)
    print(f"Fold score (accuracy): {score}")

    from jet_ml.evaluation import save_training_history
    save_training_history(history=history,fold=fold)

    from jet_ml.evaluation import plot_training_history
    plot_training_history(history=history,fold=fold)

from jet_ml.evaluation import save_training_stats
save_training_stats(accuracies=folds_accuracy,
                    epochs_needed=epochs_needed,
                    times_taken=times_taken)

# # Build the oos prediction list and calculate the error.
out_of_sample_y=np.concatenate(out_of_sample_y)
out_of_sample_pred=np.concatenate(out_of_sample_pred)

out_of_sample_y_compare=np.argmax(out_of_sample_y,axis=1)# For accuracy and confusion matrix calculation